# Modelling and simulating COVID-19 using spatial models

<p class="pm-node nj-authors">Marc Sturrock, RCSI [marcsturrock@rcsi.com]</p>

Here we expand upon the simple ODE based SIR model in the previous notebook to consider agent based spatial models. We make use of the julia package "Agents.jl" for this. Note that these models are also stochastic, so 

Here is what will be covered in this session:

- [x] Agent based models description 
- [x] Moving agents (people) in space
- [x] People bumping into each other: incorporating billiard-like interactions
- [x] Adding COVID-19 spread using SIR model dynamics
- [x] Simulating social distancing

# Model Description: Agent Based Model (ABM)

An agent-based (or individual-based) model is a computational simulation of autonomous agents that react to their environment (including other agents) given a predefined set of rules. ABMs have been adopted and studied in a variety of research disciplines. One reason for their popularity is that they enable a relaxation of many simplifying assumptions usually made by mathematical models.

Agent-based models are increasingly recognised as a useful approach for studying complex systems. Complex systems cannot be fully understood using traditional mathematical tools which aggregate the behaviour of elements in a system. The behavior of a complex system depends on both the behavior of and interactions between its elements (agents). Small changes in the input to complex systems or the behaviour of its agents can lead to large changes in outcome. That is to say, a complex system's behaviour is nonlinear, and that it is not only the sum of the behaviour of its elements. Use of ABMs have become feasible after the availability of computers and has been growing ever since, especially in modelling biological and economical systems, and has extended to social studies and archaeology.

An ABM consists of autonomous agents that behave given a set of rules. A classic example of an ABM is Schelling's segregation model which uses a regular grid and defines agents at random positions on the grid. Agents can be from different social groups. Agents are happy/unhappy based on the fraction of their neighbours that belong to the same group as they are. If they are unhappy, they keep moving to new locations until they are happy. Schelling's model shows that even small preferences of agents to have neighbours belonging to the same group (e.g. preferring that at least 30% of neighbours to be in the same group) could lead to total segregation of neighbourhoods. This is an example of emergent behaviour from simple interactions of agents that can only be captured in an agent-based model.

# Moving agents (people) in space

In [ ]:
using Agents, Random

mutable struct Agent <: AbstractAgent
    id::Int
    pos::NTuple{2,Float64}
    vel::NTuple{2,Float64}
    mass::Float64
end

The mass field will come in handy later on, when we implement social isolation (i.e. that some agents don't move and can't be moved).

Let's also initialise a trivial model with continuous space

In [ ]:
function ball_model(; speed = 0.002)
    space2d = ContinuousSpace((1, 1), 0.02)
    model = ABM(Agent, space2d, properties = Dict(:dt => 1.0), rng = MersenneTwister(42))

    # And add some agents to the model
    for ind in 1:500
        pos = Tuple(rand(model.rng, 2))
        vel = sincos(2π * rand(model.rng)) .* speed
        add_agent!(pos, model, vel, 1.0)
    end
    return model
end

model = ball_model()

We took advantage of the functionality of add_agent! that creates the agents automatically. For now all agents have the same absolute speed, and mass.

The agent step function for now is trivial. It is just move_agent! in continuous space

In [ ]:
agent_step!(agent, model) = move_agent!(agent, model, model.dt)

In [ ]:
using InteractiveDynamics
using CairoMakie
using Base64
abm_video(
    "socialdist1.mp4",
    model,
    agent_step!;
    title = "Ball Model",
    frames = 50,
    spf = 2,
    framerate = 25,
)
function showanim(filename)
    base64_video = base64encode(open(filename))
    display("text/html", """<video controls src="data:video/x-m4v;base64,$base64_video">""")
end
showanim("socialdist1.mp4")

As you can see the agents move in a straight line (with periodic boundary conditions - if an agent leaves the right boundary they re-emerge in the left boundary). There are no interactions yet.

# People bumping into each other: incorporating billiard-like interactions

We will model the people as balls that bump into each other.

In [ ]:
function model_step!(model)
    for (a1, a2) in interacting_pairs(model, 0.012, :nearest)
        elastic_collision!(a1, a2, :mass)
    end
end

model2 = ball_model()

abm_video(
    "socialdist2.mp4",
    model2,
    agent_step!,
    model_step!;
    title = "Billiard-like",
    frames = 50,
    spf = 2,
    framerate = 25,
)
showanim("socialdist2.mp4")

## Adding COVID-19 spread using SIR model dynamics
We now add more functionality to these agents, according to the SIR model (see previous example).
They can be infected with COVID-19 and transfer COVID-19 to other agents around them.

In [ ]:
mutable struct PoorSoul <: AbstractAgent
    id::Int
    pos::NTuple{2,Float64}
    vel::NTuple{2,Float64}
    mass::Float64
    days_infected::Int  # number of days since is infected
    status::Symbol  # :S, :I or :R
    β::Float64
end

Here `β` is the transmission probability, which we choose to make an
agent parameter instead of a model parameter. It reflects the level of hygiene
of an individual. In a realistic scenario, the actual virus transmission
would depend on the `β` value of both agents, but we don't do that here for simplicity.

We also significantly modify the model creation, to have SIR-related parameters.
Each step in the model corresponds to one hour.

In [ ]:
const steps_per_day = 24

using DrWatson: @dict
function sir_initiation(;
    infection_period = 30 * steps_per_day,
    detection_time = 14 * steps_per_day,
    reinfection_probability = 0.05,
    isolated = 0.0, # in percentage
    interaction_radius = 0.012,
    dt = 1.0,
    speed = 0.002,
    death_rate = 0.044, # from website of WHO
    N = 1000,
    initial_infected = 5,
    seed = 42,
    βmin = 0.4,
    βmax = 0.8,
)

    properties = @dict(
        infection_period,
        reinfection_probability,
        detection_time,
        death_rate,
        interaction_radius,
        dt,
    )
    space = ContinuousSpace((1,1), 0.02)
    model = ABM(PoorSoul, space, properties = properties, rng = MersenneTwister(seed))

    ## Add initial individuals
    for ind in 1:N
        pos = Tuple(rand(model.rng, 2))
        status = ind ≤ N - initial_infected ? :S : :I
        isisolated = ind ≤ isolated * N
        mass = isisolated ? Inf : 1.0
        vel = isisolated ? (0.0, 0.0) : sincos(2π * rand(model.rng)) .* speed

        ## very high transmission probability
        ## we are modelling close encounters after all
        β = (βmax - βmin) * rand(model.rng) + βmin
        add_agent!(pos, model, vel, mass, 0, status, β)
    end

    return model
end

Notice the constant `steps_per_day`, which approximates how many model steps
correspond to one day (since the parameters we used in the previous graph SIR example
were given in days).

To visualise this model, we will use black color for the susceptible, red for
the infected infected and green for the recovered, leveraging
[`InteractiveDynamics.abm_plot`](@ref).

In [ ]:
sir_model = sir_initiation()

sir_colors(a) = a.status == :S ? "#2b2b33" : a.status == :I ? "#bf2642" : "#338c54"

fig, abmstepper = abm_plot(sir_model; ac = sir_colors)
fig # display figure

We have increased the size of the model 10-fold (for more realistic further analysis)

To actually spread the virus, we modify the `model_step!` function,
so that individuals have a probability to transmit the disease as they interact.

In [ ]:
function transmit!(a1, a2, rp)
    ## for transmission, only 1 can have the disease (otherwise nothing happens)
    count(a.status == :I for a in (a1, a2)) ≠ 1 && return
    infected, healthy = a1.status == :I ? (a1, a2) : (a2, a1)

    rand(model.rng) > infected.β && return

    if healthy.status == :R
        rand(model.rng) > rp && return
    end
    healthy.status = :I
end

function sir_model_step!(model)
    r = model.interaction_radius
    for (a1, a2) in interacting_pairs(model, r, :nearest)
        transmit!(a1, a2, model.reinfection_probability)
        elastic_collision!(a1, a2, :mass)
    end
end

Notice that it is not necessary that the transmission interaction radius is the same
as the billiard-ball dynamics. We only have them the same here for convenience,
but in a real model they will probably differ.

We also modify the `agent_step!` function, so that we keep track of how long the
agent has been infected, and whether they have to die or not.


In [ ]:
function sir_agent_step!(agent, model)
    move_agent!(agent, model, model.dt)
    update!(agent)
    recover_or_die!(agent, model)
end

update!(agent) = agent.status == :I && (agent.days_infected += 1)

function recover_or_die!(agent, model)
    if agent.days_infected ≥ model.infection_period
        if rand(model.rng) ≤ model.death_rate
            kill_agent!(agent, model)
        else
            agent.status = :R
            agent.days_infected = 0
        end
    end
end
nothing # hide

# Alright, now we can animate this process for default parameters

sir_model = sir_initiation()

abm_video(
    "socialdist3.mp4",
    sir_model,
    sir_agent_step!,
    sir_model_step!;
    title = "SIR model",
    frames = 100,
    ac = sir_colors,
    as = 10,
    spf = 1,
    framerate = 20,
)
showanim("socialdist3.mp4")

We can also view how the time series graph looks to see if the overall number of infected varies.

In [ ]:
infected(x) = count(i == :I for i in x)
recovered(x) = count(i == :R for i in x)
adata = [(:status, infected), (:status, recovered)]
nothing # hide

# Let's do the following runs, with different parameters probabilities
r1, r2 = 0.04, 0.33
β1, β2 = 0.5, 0.1
sir_model1 = sir_initiation(reinfection_probability = r1, βmin = β1)
sir_model2 = sir_initiation(reinfection_probability = r2, βmin = β1)
sir_model3 = sir_initiation(reinfection_probability = r1, βmin = β2)

data1, _ = run!(sir_model1, sir_agent_step!, sir_model_step!, 2000; adata)
data2, _ = run!(sir_model2, sir_agent_step!, sir_model_step!, 2000; adata)
data3, _ = run!(sir_model3, sir_agent_step!, sir_model_step!, 2000; adata)

data1[(end-10):end, :]
@show size(data1)
# Now, we can plot the number of infected versus time
using CairoMakie
figure = Figure()
ax = figure[1, 1] = Axis(figure; ylabel = "Infected")
l1 = lines!(ax, data1[:, 2], color = :orange)
l2 = lines!(ax, data2[:, 2], color = :blue)
l3 = lines!(ax, data3[:, 2], color = :green)
figure[1, 2] =
    Legend(figure, [l1, l2, l3], ["r=$r1, beta=$β1", "r=$r2, beta=$β1", "r=$r1, beta=$β2"])
figure

# Simulating social distancing
Of course we are beyond the point of socially distancing and rolling out vaccinations. This effectively introduces a 4th type of status, :V for vaccinated. This type can't get infected, and thus all remaining individuals that are already infected will (hopefully) survive or die out.

Until that point, social distancing is practiced. The best way to model social distancing is to make some agents simply not move. You can think of this as people working from home/not travelling as much. Of course not everyone obeys these rules - try playing with the fraction of isolated (socially distancing) people below. 

In [ ]:
sir_model = sir_initiation(isolated = 0.8)
abm_video(
    "socialdist4.mp4",
    sir_model,
    sir_agent_step!,
    sir_model_step!;
    title = "Social Distancing",
    frames = 100,
    spf = 2,
    ac = sir_colors,
    framerate = 20,
)
showanim("socialdist4.mp4")

Here we let some 20% of the population not be isolated, this simulates those in the population who do not follow government guidelines. Still, you can see that the spread of the virus is still dramatically welll contained.

Rather than trying to eye-ball animations and guess which scenarios are better, we can look at the related time-series graphs too.

In [ ]:
r4 = 0.04
sir_model4 = sir_initiation(reinfection_probability = r4, βmin = β1, isolated = 0.95)

data4, _ = run!(sir_model4, sir_agent_step!, sir_model_step!, 2000; adata)

l4 = lines!(ax, data4[:, 2], color = :red)
figure[1, 2] = Legend(
    figure,
    [l1, l2, l3, l4],
    ["r=$r1, beta=$β1", "r=$r2, beta=$β1", "r=$r1, beta=$β2", "r=$r4, social distancing"],
)
figure

Here you can see the characteristic "flattening the curve" which avoids the health system being overwhelemed. There are many expansions available for such models, a nice overview is given here:

https://www.youtube.com/watch?v=gxAaO2rsdIs